In [ ]:
import tkinter as tk
import pandas as pd
import math
import pandas as pd
import numpy as np
import cv2

import pygetwindow as gw
import pyautogui
import threading
import time

from PIL import Image, ImageStat
from matplotlib import pyplot as plt
from os.path import exists

# for inventory item detection

In [ ]:
def edge_detection(img_np):
    img = cv2.cvtColor(img_np, cv2.COLOR_BGR2GRAY)
#     img = cv2.blur(img, (3,3))
    img = cv2.GaussianBlur(img, (3,3), 0)
    img = cv2.Canny(img, threshold1=50, threshold2=200, apertureSize=5)
#     img = cv2.blur(img, (2,2))
    img = cv2.blur(img, (3,3))
    return img

def inventory_line_detection(img):
    downscale = 2
    img = cv2.resize(img, ((int) (img.shape[1]/downscale)
                                     , (int) (img.shape[0]/downscale)))
    
    img_inventory_edge = edge_detection(img)
    
    img_horizontal = img_inventory_edge.copy()
    img_vertical = img_inventory_edge.copy()

    line_length = (int) (40/downscale)
    horizontal_struct = cv2.getStructuringElement(cv2.MORPH_RECT, (line_length,1))
    vertical_struct = cv2.getStructuringElement(cv2.MORPH_RECT, (1,line_length))

    cv2.erode(img_horizontal, horizontal_struct, img_horizontal)
    cv2.dilate(img_horizontal, horizontal_struct, img_horizontal)

    cv2.erode(img_vertical, vertical_struct, img_vertical)
    cv2.dilate(img_vertical, vertical_struct, img_vertical)

    result = cv2.add(img_horizontal, img_vertical)
    return result

def find_slot_locations(inventory_filtered, slot_gray):
    # scale down images
    downscale = 2
#     inv = cv2.resize(inventory_filtered, ((int) (inventory_filtered.shape[0]/downscale)
#                                      , (int) (inventory_filtered.shape[1]/downscale)))
    inv = inventory_filtered
    slot = cv2.resize(slot_gray, ((int) (slot_gray.shape[1]/downscale)
                                     , (int) (slot_gray.shape[0]/downscale)))
    
    matched_slots = cv2.matchTemplate(inv, slot, cv2.TM_CCORR_NORMED)
    #matched_slots = inventory_filtered
    
    threshold = 0.6
    #threshold = 100
    slot_locations = []
    slots_min_x = (int) (1920/3/downscale)
    min_distance = (int) (60/downscale)
    slot_locations_filtered = []
    
    for y in range(matched_slots.shape[0]):
        for x in range(matched_slots.shape[1]):
            if matched_slots[y][x] > threshold:
                if x > slots_min_x:
                    slot_locations.append((x,y))
    
    # filter number of locations
    for x,y in slot_locations:
        add_slot = True
        for s in slot_locations_filtered:
            if abs(s[0]-x) < min_distance and abs(s[1]-y) < min_distance:
                add_slot = False
                break
        if add_slot == True:
            slot_locations_filtered.append((x,y))
    
    # upscale slot locations
    for i in range(len(slot_locations_filtered)):
        x,y = slot_locations_filtered[i]
        slot_locations_filtered[i] = (x*downscale, y*downscale)
            
    return slot_locations_filtered,matched_slots

def draw_slots_on_image(img, slots):
    img_with_slots = img.copy()
    for x,y in slots:
        img_with_slots = cv2.circle(img_with_slots, (x, y), radius=5, color=(255,0,0), thickness=-1)
    return img_with_slots

def get_items_from_inventory(inventory, slots):
    global slot_size
    
    items = []
    for s in slots:
        items.append(inventory[s[1]:s[1]+slot_size, s[0]:s[0]+slot_size])
        
    return items

# to predict the item image

In [ ]:
def load_icons_from_disk(verbose=False):
    global all_items_df
    
    icons = []
    for index,item in all_items_df.iterrows():
        filename = path_grid_icons + item['id'] + filename_ending_grid_icon
        if exists(filename):
            icon = np.asarray(Image.open(filename))
            icon = cv2.resize(icon, ((int) (icon.shape[1]*window_scale_factor)
                                     , (int) (icon.shape[0]*window_scale_factor)))
            icons.append(icon)
        else:
            icons.append([])
            if verbose:
                print(f"File {filename} does not exist.")
    return icons

def run_sift(img):
    global sift
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
#     edge = cv2.Canny(gray, threshold1=50, threshold2=50, apertureSize=5)
#     gray = cv2.blur(gray, (2,2))
    kp,des = sift.detectAndCompute(gray,None)
    return kp,des

def run_orb(img):
    global orb
    kp,des = orb.detectAndCompute(img, None)
    return kp,des

def run_surf(img):
    global surf
    kp,des = surf.detectAndCompute(img,None)
    return kp,des
    
def create_all_descriptors():
    global icons
    descriptors = []
    for icon in icons:
        if len(icon) == 0:
            descriptors.append(None)
            continue
        icon = icon[0:slot_size, 0:slot_size]
        kp,des = run_sift(icon) #run_orb(icon) #run_sift(icon)
        descriptors.append(des)
    return descriptors

def predict_icon(img):
    global icons, bf, descriptors
    distances = []

    kp,des = run_sift(img) #run_orb(img) #run_sift(img)

    for i in range(len(icons)):
        if descriptors[i] is None:
            distances.append(999999)
            continue

        matches = bf.match(des,descriptors[i])
        matches = sorted(matches, key = lambda x:x.distance)
        matches = matches[:30]
        
        if len(matches) == 0:
            distances.append(999999)
            continue
        
#         print(f'i: {i} -- {matches}')
        distance = 0.0
        for match in matches:
            distance += match.distance/len(matches)
            
        distances.append(distance)

    prediction = np.argmin(distances)
    min_distance = min(distances)
    
#     print(f"distance: {min_distance}")
    
    # filter predictions
#     threshold = 60000
#     if min_distance > threshold:
#         return -1,-1
    
    return prediction,min_distance

def predict_all_icons(images):
    predictions = []
    distances = []
    for item in images:
        p,d = predict_icon(item)
        predictions.append(p)
        distances.append(d)
    return predictions,distances

# find items in inventory

In [ ]:
# paths
path_icons = './icons/'
path_images = './images/'
path_grid_icons = './grid_icons/'
path_data = './data/'

filename_ending_grid_icon = '-grid-image.jpg'

matched_slots = []

In [ ]:
# get tarkov window
window_title_tarkov = 'EscapeFromTarkov'
window_tarkov = gw.getWindowsWithTitle(window_title_tarkov)[0]
window_tarkov_position = window_tarkov.topleft
window_tarkov_size = window_tarkov.size

window_scale_factor = window_tarkov_size[1] / 1080.0
slot_size = (int) (64*window_scale_factor) # in pixels, for FHD resolution

In [ ]:
# load inventory image
# img_inventory = Image.open(path_images + 'inventar4.png')
# img_inventory = np.asarray(img_inventory)
screenshot = pyautogui.screenshot(region=(window_tarkov_position[0],window_tarkov_position[1]
                                              , window_tarkov_size[0], window_tarkov_size[1]))
img_inventory = np.array(screenshot)

# filter inventory image
t0 = time.time()
inventory_filtered = inventory_line_detection(img_inventory)
print(f'took {time.time()-t0} s')

In [ ]:
# load and filter slot reference image
img_slot = Image.open(path_images + 'slot_empty.png')
img_slot = np.asarray(img_slot)
img_slot = cv2.resize(img_slot, ((int) (img_slot.shape[0]*window_scale_factor), (int) (img_slot.shape[1]*window_scale_factor)))
img_slot_gray = cv2.cvtColor(img_slot, cv2.COLOR_BGR2GRAY)

In [ ]:
# find slots in inventory
t0 = time.time()
slot_locations, matched_slots = find_slot_locations(inventory_filtered, img_slot_gray)
print(f'took {time.time()-t0} s')

# draw slots on image
img_inventory = draw_slots_on_image(img_inventory, slot_locations)

In [ ]:
# get item images from inventory
item_images = get_items_from_inventory(img_inventory, slot_locations)

In [ ]:
len(item_images)

# predict items

In [ ]:
icons = []
descriptors = []
all_items_df = None
predictions = []
distances = []
sift = cv2.SIFT_create()
orb = cv2.ORB_create(nfeatures=100)
#surf = cv2.xfeatures2d.SURF_create(400)
# bf = cv2.BFMatcher(cv2.NORM_L1, crossCheck=False)
bf = cv2.BFMatcher()

# get descriptors of all items
all_items_df = pd.read_csv(path_data + 'all_items.csv')
icons = load_icons_from_disk()
descriptors = create_all_descriptors()

In [ ]:
len(icons)

In [ ]:
# predict each item from inventory
predictions,distances = predict_all_icons(item_images)
predictions_df = pd.DataFrame({'slot_x': 0, 'slot_y': 0, 'predicted_item': predictions, 'distance': distances})

In [ ]:
for i in range(len(predictions_df)):
    predictions_df.loc[i,'slot_x'] = slot_locations[i][0]
    predictions_df.loc[i,'slot_y'] = slot_locations[i][1]

In [ ]:
# save predictions to file
# predictions_df.to_csv(path_data + 'predictions_inventar4.csv', index=False)

# debugging stuff

In [ ]:
# item_images

In [ ]:
# threshold = 200000
# predictions_filtered = []
# for prediction in predictions:
#     if prediction[1] > threshold:
#         predictions_filtered.append((0, -1))
#     else:
#         predictions_filtered.append(prediction)

In [ ]:
predictions_here = predictions

w = 10
h = 10
fig = plt.figure(figsize=(8, 8))
columns = 10
rows = 10
for i in range(0,columns*rows,2):
    img = item_images[(int) (i/2)].copy()
    fig.add_subplot(rows, columns, i+1)
    plt.imshow(img)
    img = icons[predictions[(int) (i/2)]]
    fig.add_subplot(rows, columns, i+2)
    plt.imshow(img)
plt.show()

In [ ]:
w = 10
h = 10
fig = plt.figure(figsize=(8, 14))
columns = 5
rows = 10
for i in range(0,columns*rows,1):
#     if distances[(int) (i/2)]:
#         continue
    img1 = item_images[i].copy()
    kp1,des1 = run_sift(img1)
    
    img2 = icons[predictions[i]].copy()
    kp2,des2 = run_sift(img2)
    
    matches = bf.match(des1,des2)
    matches = sorted(matches, key = lambda x:x.distance)
    matches = matches[:5]
                
    img3 = cv2.drawMatches(img1, kp1, img2, kp2, matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    
    distance = (int) (distances[i])
    if distance > 300:
        continue
        
    fig.add_subplot(rows, columns, i+1, title=distance)
    plt.imshow(img3)
    
plt.show()

In [ ]:
def run_sift(img):
    global sift
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
#     edge = cv2.Canny(gray, threshold1=50, threshold2=50)
#     gray = cv2.blur(gray, (3,3))
    kp,des = sift.detectAndCompute(gray,None)
    return kp,des

In [ ]:
i = 200
img1 = icons[i].copy()
img1 = cv2.resize(img1, ((int) (img1.shape[0]*window_scale_factor), (int) (img1.shape[1]*window_scale_factor)))
kp1,des1 = run_sift(img1)
img1 = cv2.drawKeypoints(img1,kp1,img1,flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

Image.fromarray(img1)

In [ ]:
i = 2
img2 = item_images[i].copy()
kp2,des2 = run_sift(img2)
img2 = cv2.drawKeypoints(img2,kp2,img2,flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

Image.fromarray(img2)

In [ ]:
matches = bf.match(des1,des2)
# Apply ratio test
# good = []
# for m,n in matches:
#     if m.distance < 0.75*n.distance:
#         good.append([m])
matches = sorted(matches, key = lambda x:x.distance)
matches = matches[:20]
                
img3 = cv2.drawMatches(img1, kp1, img2, kp2, matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
Image.fromarray(img3)

In [ ]:
distance = 0.0
for match in matches:
    distance += match.distance/len(matches)
    
print(distance)

In [ ]:
predictions

In [ ]:
Image.fromarray(inventory_filtered)

In [ ]:
# Image.fromarray(img_slot_np_edge)
# Image.fromarray(res)
# plt.imshow(res[3], cmap='gray')
Image.fromarray(img_inventory)
# Image.fromarray(hough)

In [ ]:
downscale = 2
img = img_inventory.copy()
img = cv2.resize(img, ((int) (img.shape[1]/downscale)
                                 , (int) (img.shape[0]/downscale)))

img_inventory_edge = edge_detection(img)

img_horizontal = img_inventory_edge.copy()
img_vertical = img_inventory_edge.copy()

line_length = (int) (40/downscale)
horizontal_struct = cv2.getStructuringElement(cv2.MORPH_RECT, (line_length,1))
vertical_struct = cv2.getStructuringElement(cv2.MORPH_RECT, (1,line_length))

cv2.erode(img_horizontal, horizontal_struct, img_horizontal)
cv2.dilate(img_horizontal, horizontal_struct, img_horizontal)

cv2.erode(img_vertical, vertical_struct, img_vertical)
cv2.dilate(img_vertical, vertical_struct, img_vertical)

result = cv2.add(img_horizontal, img_vertical)

In [ ]:
img.shape

In [ ]:
Image.fromarray(result)